# Intraclass Correlation Coefficient (ICC)

In [7]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_theme(
    context="paper", 
    style="whitegrid", 
    font_scale=1.2,
    rc={'figure.figsize': (10, 10), 'figure.dpi': 300}
)

import pingouin as pg
from common.data import get_model_weights
from common.wisc import WISC_LEVEL
from common.binning import BIN_LABELS
from common.paths import MODEL_RESULTS, ICC_RESULTS
from common.results import save_results

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Between PLS and Ridge

In [2]:
%%time
results = []
population = 'adhd'

for bin_label in BIN_LABELS:
    for target in WISC_LEVEL[5]:
        pls_weights = get_model_weights('pls', population, target, bin_label)
        ridge_weights = get_model_weights('ridge', population, target, bin_label)
        
        icc_data = pd.DataFrame([pls_weights, ridge_weights]).melt(
            var_name='connection', value_name='weight', ignore_index=False)
        icc_data['cv_run_num'] = icc_data.index
        icc = pg.intraclass_corr(data=icc_data, targets='connection', raters='cv_run_num', ratings='weight').round(3)
        icc1 = icc[icc['Type'] == 'ICC1'].to_dict('records')[0]
        
        results.append({
            'Population': population,
            'Target': target,
            'Bin': bin_label,
            **icc1,
        })
#         print(results[-1])
        
results_df = pd.DataFrame(results)
display(results_df)

,Population,Target,Bin,Type,Description,ICC,F,df1,df2,pval,CI95%
0,adhd,WISC_FSIQ,All,ICC1,Single raters absolute,0.959,47.896,34715,34716,0.0,"[0.96, 0.96]"
1,adhd,WISC_VSI,All,ICC1,Single raters absolute,0.961,49.857,34715,34716,0.0,"[0.96, 0.96]"
2,adhd,WISC_VCI,All,ICC1,Single raters absolute,0.960,48.799,34715,34716,0.0,"[0.96, 0.96]"
3,adhd,WISC_FRI,All,ICC1,Single raters absolute,0.959,47.528,34715,34716,0.0,"[0.96, 0.96]"
4,adhd,WISC_WMI,All,ICC1,Single raters absolute,0.948,37.553,34715,34716,0.0,"[0.95, 0.95]"
5,adhd,WISC_PSI,All,ICC1,Single raters absolute,0.955,43.353,34715,34716,0.0,"[0.95, 0.96]"
6,adhd,WISC_FSIQ,Bin 1,ICC1,Single raters absolute,0.974,76.008,34715,34716,0.0,"[0.97, 0.97]"
7,adhd,WISC_VSI,Bin 1,ICC1,Single raters absolute,0.949,38.133,34715,34716,0.0,"[0.95, 0.95]"
8,adhd,WISC_VCI,Bin 1,ICC1,Single raters absolute,0.968,61.623,34715,34716,0.0,"[0.97, 0.97]"
9,adhd,WISC_FRI,Bin 1,ICC1,Single raters absolute,0.985,135.524,34715,34716,0.0,"[0.99, 0.99]"


CPU times: user 12min 35s, sys: 904 ms, total: 12min 36s
Wall time: 12min 36s


In [4]:
fn = f'between-model_icc_{population}'
save_results(results_df, fn, ICC_RESULTS)

'/home/bpho/Documents/MSc_Research-Project/scratch_data/model_results/ICC/between-model_icc_adhd.csv'

## Between age bins

In [3]:
import itertools
from common.power_atlas import to_power_network_fc_vector

all_bin_combinations = list(itertools.combinations(BIN_LABELS, 2))
print(all_bin_combinations)

[('All', 'Bin 1'), ('All', 'Bin 2'), ('All', 'Bin 3'), ('Bin 1', 'Bin 2'), ('Bin 1', 'Bin 3'), ('Bin 2', 'Bin 3')]


In [7]:
%%time
results = []
population = 'adhd'
model = 'ridge'

for target in WISC_LEVEL[5]:
    for bin_pair in all_bin_combinations:
        first_bin, second_bin = bin_pair
        first_bin_weights = get_model_weights(model, population, target, first_bin)
        second_bin_weights = get_model_weights(model, population, target, second_bin)
        
        first_bin_weights = to_power_network_fc_vector(first_bin_weights)
        second_bin_weights = to_power_network_fc_vector(second_bin_weights)
        
        icc_data = pd.DataFrame([first_bin_weights, second_bin_weights]).melt(
            var_name='connection', value_name='weight', ignore_index=False)
        icc_data['cv_run_num'] = icc_data.index
        icc = pg.intraclass_corr(data=icc_data, targets='connection', raters='cv_run_num', ratings='weight').round(3)
        icc1 = icc[icc['Type'] == 'ICC1'].to_dict('records')[0]
        
        results.append({
            'Model': model,
            'Population': population,
            'Target': target,
            'First Bin': first_bin,
            'Second Bin': second_bin,
            **icc1,
        })
#         print(results[-1])
        
results_df = pd.DataFrame(results)
display(results_df)

,Model,Population,Target,First Bin,Second Bin,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ridge,adhd,WISC_FSIQ,All,Bin 1,ICC1,Single raters absolute,0.559,3.532,90,91,0.000,"[0.4, 0.68]"
1,ridge,adhd,WISC_FSIQ,All,Bin 2,ICC1,Single raters absolute,0.455,2.667,90,91,0.000,"[0.28, 0.6]"
2,ridge,adhd,WISC_FSIQ,All,Bin 3,ICC1,Single raters absolute,0.666,4.996,90,91,0.000,"[0.53, 0.77]"
3,ridge,adhd,WISC_FSIQ,Bin 1,Bin 2,ICC1,Single raters absolute,0.050,1.105,90,91,0.317,"[-0.16, 0.25]"
4,ridge,adhd,WISC_FSIQ,Bin 1,Bin 3,ICC1,Single raters absolute,0.097,1.215,90,91,0.178,"[-0.11, 0.3]"
5,ridge,adhd,WISC_FSIQ,Bin 2,Bin 3,ICC1,Single raters absolute,0.125,1.286,90,91,0.116,"[-0.08, 0.32]"
6,ridge,adhd,WISC_VSI,All,Bin 1,ICC1,Single raters absolute,0.420,2.449,90,91,0.000,"[0.24, 0.58]"
7,ridge,adhd,WISC_VSI,All,Bin 2,ICC1,Single raters absolute,0.543,3.374,90,91,0.000,"[0.38, 0.67]"
8,ridge,adhd,WISC_VSI,All,Bin 3,ICC1,Single raters absolute,0.582,3.788,90,91,0.000,"[0.43, 0.7]"
9,ridge,adhd,WISC_VSI,Bin 1,Bin 2,ICC1,Single raters absolute,0.068,1.146,90,91,0.259,"[-0.14, 0.27]"


CPU times: user 11.1 s, sys: 22.8 ms, total: 11.1 s
Wall time: 11.1 s


In [8]:
filename = f'between-age-bins_icc_{population}_network_level'
save_results(results_df, filename, ICC_RESULTS)

'/home/bpho/Documents/MSc_Research-Project/scratch_data/model_results/ICC/between-age-bins_icc_adhd_network_level.csv'